In [ ]:
# -----------------------------导入python库------------------------------------
import os
import netCDF4 as nc
import pandas as pd
import numpy as np
import numpy.ma as ma
import xarray as xr
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.style as mplstyle
import cartopy.crs as ccrs
import netCDF4 as nc
from netCDF4 import Dataset
import shapefile
import time

# 变量名转换表
var_dict = {
    
}
var_list = ['Prec', 'ET', 'LH', 'SH', 'SSM', 'SST', 'Q_2m', 'T_2m', 'PSFC', 'Wind_10m']
sim_folder_mapping = {
    'ET': 'ET',
    'LH': 'LH',
    'SH': 'SH',
    'SSM': 'SH2O_Layer1',  # 土壤湿度第一层
    'SST': 'TSLB_Layer1',  # 土壤温度第一层
    'Q_2m': 'Q2',
    'T_2m': 'T2',
    'PSFC': 'PSFC',
    'Wind_10m': 'Wind10',
    'Prec':'RAINNC',
}
sim_var_mapping = {
    'ET': 'ET',
    'LH': 'LH',
    'SH': 'SH',
    'SSM': 'SH2O_Layer1',  # 土壤湿度第一层
    'SST': 'TSLB_Layer1',  # 土壤温度第一层
    'Q_2m': 'Q2',
    'T_2m': 'T2',
    'PSFC': 'PSFC',
    'Wind_10m': 'Wind10',
    'Prec': 'RAINNC',
}
obs_folder_mapping = {
    'ET': 'ET',
    'LH': 'LH',
    'SH': 'SH',
    'SSM': 'ERA5_Land',  # 土壤湿度第一层
    'SST': 'LST',  # 土壤温度第一层
    'Q_2m': 'shum',
    'T_2m': 'temp',
    'PSFC': 'pres',
    'Wind_10m': 'wind',
    'Prec': 'CHM_PRE',
}
obs_var_mapping = {
    'ET': 'ET',
    'LH': 'LH',
    'SH': 'SH',
    'SSM': 'ERA5_Land',  # 土壤湿度第一层
    'SST': 'LST',  # 土壤温度第一层
    'Q_2m': 'shum',
    'T_2m': 'temp',
    'PSFC': 'pres',
    'Wind_10m': 'wind',
    'Prec': 'Prec',
}
result_dict = {
    'Obs': {},
    'Sim': {},
            }
DataPath = '/home/yfdong/data/work/LF-SAM/output/evaluation/9km'



In [ ]:
import numpy as np
from netCDF4 import Dataset
def create_domain_mask(lat, lon, domain):
   domain_lat_min, domain_lat_max, domain_lon_min, domain_lon_max = domain
   mask = (lat >= domain_lat_min) & (lat <= domain_lat_max) & (lon >= domain_lon_min) & (lon <= domain_lon_max)
   masked_array = xr.DataArray(np.where(mask, 1, np.nan), dims=['south_north', 'west_east'])
   return masked_array
def extract_SubsetRegion_mask(var ,domain_mask):
    row_indices, col_indices = np.where(domain_mask == 1)  # 提取 ==1 的区域
    row_start, row_end = row_indices.min(), row_indices.max()
    col_start, col_end = col_indices.min(), col_indices.max()
    # 步骤2：切片获取子区域数据（保持二维结构）
    sub_var = var[row_start:row_end+1, col_start:col_end+1]
    return sub_var
# 文件路径
GeoPath = '/home/yfdong/data/work/LF-SAM/output/evaluation/9km/CHN_domain_9km.nc'

# 打开 NetCDF 文件
dst_nc = Dataset(GeoPath)

# 读取变量
lat_var = dst_nc.variables['lat'][:]
lon_var = dst_nc.variables['lon'][:]
mask = dst_nc.variables['Band1'][:]
lat_grid, lon_grid = np.meshgrid(lat_var, lon_var)

# 将 mask 数组转换为浮点类型
mask = mask.astype(float)
# 将 0 替换为 NaN
mask[mask == 0] = np.nan
print(lat_var.shape)
# 指定范围
DomainRegion2 = [25, 35, 109, 122]
lat_min, lat_max, lon_min, lon_max = DomainRegion2

# 找到指定范围内的索引
lat_indices = np.where((lat_var >= lat_min) & (lat_var <= lat_max))[0]
lon_indices = np.where((lon_var >= lon_min) & (lon_var <= lon_max))[0]

# 创建一个与 mask 形状相同的全 NaN 数组
mask_nan = np.full_like(mask, np.nan)

# 将指定范围内的值复制到 mask_nan 中
mask_nan[lat_indices.min():lat_indices.max()+1, lon_indices.min():lon_indices.max()+1] = mask[lat_indices.min():lat_indices.max()+1, lon_indices.min():lon_indices.max()+1]
# plt.imshow(mask_nan)
# 假设 num_times 是时间步数
num_times = 1  # 请根据实际情况设置时间步数

# 创建一个三维数组，形状为 (num_times, lat.shape[0], lon.shape[0])
mask_3d = mask_nan[np.newaxis, :, :].repeat(1, axis=0)
# plt.imshow(mask_3d[0,:])
# # 打印结果以验证
# print("Shape of mask_3d:", mask_3d.shape)
# print("First time step of mask_3d:\n", mask_3d[0])

# 关闭 NetCDF 文件
dst_nc.close()


In [ ]:

# -------------------------- 数据预处理模块 ------------------------------

# for EXP_TYPE in ['WCTRL', 'HCTRL']:
for year in [2011,2018,2020]:
    start_date = f'{year}-06-01 00:00:00'
    end_date = f'{year}-8-30 23:59:59'     
    # 生成时间序列，间隔为6小时
    dates = pd.date_range(start=start_date, end=end_date, freq='24H') 
    # 生成唯一的缓存文件名（包含关键参数）
    cache_filename = (
        f"LSMvar_evaluation_JJA_Daily_{len(var_list)}vars"
        f"_{year}.nc"
    )
    cache_path = os.path.join(DataPath, 'cache', cache_filename)
    if os.path.exists(cache_path):
        print(f"缓存文件已存在: {cache_path}")
        # with nc.Dataset(cache_path, 'r') as ds:
        #     # 读取结果数据
        #     results = {
        #         'Ctrl': {var: ds[f'Ctrl_{var}'][:] for var in var_list},
        #         'Expl': {var: ds[f'Expl_{var}'][:] for var in var_list},
        #     }
    else:
        print("未找到缓存文件，重新处理数据...")
        start = time.time()
        # 初始化结果字典
        result_dict = {'Obs': {}, 'WRF-S': {}, 'WRF-H': {}}
        for var in var_list:
            
            sim_folder = sim_folder_mapping[var]
            obs_folder = obs_folder_mapping[var]
            sim_var_name = sim_var_mapping[var]
            obs_var_name = obs_var_mapping[var]
            temp_WRF_S_var = np.zeros((len(dates), lat_var.shape[0], lon_var.shape[0]))
            temp_WRF_H_var = np.zeros((len(dates), lat_var.shape[0], lon_var.shape[0]))
            temp_obs_var = np.zeros((len(dates), lat_var.shape[0], lon_var.shape[0]))
            print(sim_folder, obs_folder)
            for TimeIndex, date in enumerate(dates):
                WRF_S_file = os.path.join(DataPath, 'sim', sim_folder, f'{date.strftime("%Y%m%d")}.WCTRL_{sim_folder}') 
                WRF_H_file = os.path.join(DataPath, 'sim', sim_folder, f'{date.strftime("%Y%m%d")}.HCTRL_{sim_folder}') 
                if var == 'Prec':
                    obs_file = os.path.join(DataPath, 'obs', obs_folder, 'output', f'{date.strftime("%Y%m%d")}.{obs_folder}') 
                else:
                    obs_file = os.path.join(DataPath, 'obs', obs_folder, 'output', f'{date.strftime("%Y%m%d")}00.{obs_folder}')
                # 使用上下文管理器打开NetCDF文件
                with nc.Dataset(WRF_S_file, 'r') as WRF_S_nc:
                    # 直接读取指定时间索引的数据，并写入目标数组
                    temp_WRF_S_var[TimeIndex, :, :] = WRF_S_nc.variables[sim_var_name][:] 
                with nc.Dataset(WRF_H_file, 'r') as WRF_H_nc:
                    temp_WRF_H_var[TimeIndex, :, :] = WRF_H_nc.variables[sim_var_name][:] 
                with nc.Dataset(obs_file, 'r') as obs_data:
                    # 同样地，直接读取并写入
                    temp_obs_var[TimeIndex, :, :] = obs_data.variables[obs_var_name][:] 
                    
            result_dict['WRF-S'][var] = temp_WRF_S_var *mask_3d
            result_dict['WRF-H'][var] = temp_WRF_H_var *mask_3d
            result_dict['Obs'][var] = temp_obs_var *mask_3d
        # ===================== 保存结果到缓存文件 =====================
        os.makedirs(os.path.dirname(cache_path), exist_ok=True)
        with nc.Dataset(cache_path, 'w') as ds:
            # 创建维度
            ds.createDimension('lat', lat_var.shape[0])
            ds.createDimension('lon', lon_var.shape[0])
            ds.createDimension('Date', result_dict['WRF-S'][var].shape[0])
            # 保存坐标变量
            lat = ds.createVariable('lat', 'f4', ('lat'))
            lon = ds.createVariable('lon', 'f4', ('lon'))
            Date = ds.createVariable('Date', 'f4', ('Date'))
            lat[:] = lat_var[:]
            lon[:] = lon_var[:]
            Date[:] = np.arange(0, result_dict['WRF-S'][var].shape[0])
            # 保存计算结果
            for var in var_list:
                print(var)
                WRF_S_var = ds.createVariable(f'WRF-S_{var}', 'f4', ('Date', 'lat', 'lon'), zlib =True)
                WRF_H_var = ds.createVariable(f'WRF-H_{var}', 'f4', ('Date', 'lat', 'lon'), zlib =True)
                obs_var = ds.createVariable(f'Obs_{var}', 'f4', ('Date', 'lat', 'lon'), zlib =True)
                WRF_S_var[:] = result_dict['WRF-S'][var]
                WRF_H_var[:] = result_dict['WRF-H'][var]
                obs_var[:] = result_dict['Obs'][var]
            # 添加文件属性便于追溯
            ds.setncattr('Generated_Time', time.ctime())   


In [ ]:
# --------------------------------- 数据读取模块 ------------------------------------


# 定义年份列表
years = [2011, 2018, 2020]

# 初始化结果字典
read_result_dict = {'Obs': {}, 'WRF-S': {}, 'WRF-H': {}}

# 初始化每个变量的数据存储
for var in var_list:
    read_result_dict['WRF-S'][var] = []
    read_result_dict['WRF-H'][var] = []
    read_result_dict['Obs'][var] = []

# 处理每个年份
for year in years:
    start_date = f'{year}-06-01 00:00:00'
    end_date = f'{year}-08-30 23:59:59'
    
    # 生成时间序列，间隔为24小时
    dates = pd.date_range(start=start_date, end=end_date, freq='24H')
    
    # 构建缓存文件路径
    cache_filename = f"LSMvar_evaluation_JJA_Daily_{len(var_list)}vars_{year}.nc"
    cache_path = os.path.join(DataPath, 'cache', cache_filename)
    
    # 打开 NetCDF 文件
    with Dataset(cache_path, 'r') as ds:
        for var in var_list:
            # 读取数据
            WRF_S_data = ds[f'WRF-S_{var}'][:]
            WRF_H_data = ds[f'WRF-H_{var}'][:]
            obs_data = ds[f'Obs_{var}'][:]
            
            # 将数据添加到结果字典中
            read_result_dict['WRF-S'][var].append(WRF_S_data)
            read_result_dict['WRF-H'][var].append(WRF_H_data)
            read_result_dict['Obs'][var].append(obs_data)

# 将结果字典中的列表转换为 numpy 数组
for var in var_list:
    read_result_dict['WRF-S'][var] = np.concatenate(read_result_dict['WRF-S'][var])
    read_result_dict['WRF-H'][var] = np.concatenate(read_result_dict['WRF-H'][var])
    read_result_dict['Obs'][var] = np.concatenate(read_result_dict['Obs'][var])

# 打印结果以验证
for var in var_list:
    print(f"Variable: {var}")
    print("Sim Data Shape:", read_result_dict['WRF-H'][var].shape)
    print("Obs Data Shape:", read_result_dict['Obs'][var].shape)


In [ ]:
# --------------------------- 计算模块 (多年平均的空间评分) -------------------------------
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
def return_var_unit(var_name):
    var_unit_dict = {
        'SSM': 'm³/m³',
        'ET': 'mm/day',
        'RH_2m': '%',
        'RH_850hPa': '%',
        'CLF': '%',
        'LCL': 'm',
        'Prec': 'mm/day',
        'RUNSF': 'mm/day',
        'RUNSB': 'mm/day',
        'QVAPOR_850hPa': 'g/kg',
        'QVAPOR_2m': 'g/kg',
        'Q_2m': 'g/kg',
        'SST': 'K',
        'NLW': 'W/m2',
        'NSW': 'W/m2',
        'LH': 'W/m2',
        'SH': 'W/m2',
        'CAPE': 'J/kg',
        'PBLH': 'm',
        'T_2m': 'K',
        'PSFC': 'hPa',
        'Wind_10m': 'm/s',
        'T_850hPa': 'K'
    }
    return var_unit_dict.get(var_name, 'Unknown Unit')
# 假设result_dict是已加载的数据字典，包含'Sim'和'Obs'两个键
# var_list是变量列表
var_list = ['Prec', 'ET', 'LH', 'SH', 'SSM', 'SST', 'Q_2m', 'T_2m', 'PSFC', 'Wind_10m']
# 初始化结果DataFrame
S_Metrics_df = pd.DataFrame(index=var_list, columns=['Unit','SCORR', 'SRMSE', 'SBIAS'])
EXPs = ['WRF-S', 'WRF-H']
for EXP in EXPs:
    for var in var_list:
        S_Metrics_df.loc[var, 'Unit'] = return_var_unit(var)
        sim_data = read_result_dict[EXP][var]  # (273, 400, 689)
        obs_data = read_result_dict['Obs'][var]  # (273, 400, 689)
        # 质控
        sim_data[sim_data>10000]=np.nan
        obs_data[obs_data>10000]=np.nan
        if var == 'SSM':
            sim_data[sim_data>0.6]=np.nan
            obs_data[obs_data>0.6]=np.nan
        print(var, np.nanmax(sim_data), np.nanmax(obs_data))
        print(var, np.nanmin(sim_data), np.nanmin(obs_data))
        
        # 初始化存储每个时间步指标的列表
        spatial_corrs = []
        spatial_rmses = []
        spatial_bias = []
        sim_grid = (np.nanmean(sim_data, axis=0)).flatten()
        obs_grid = (np.nanmean(obs_data, axis=0)).flatten()
        
        # 创建有效数据掩码（排除NaN值）
        mask = ~np.isnan(sim_grid) & ~np.isnan(obs_grid)
        
        # 提取有效数据
        valid_sim = sim_grid[mask]
        valid_obs = obs_grid[mask]
        
        spatial_corrs , _ = pearsonr(valid_sim, valid_obs)
        spatial_rmses = np.sqrt(mean_squared_error(valid_obs, valid_sim))
        spatial_bias = np.nanmean(valid_sim - valid_obs)
        # 计算时间平均的空间指标
        S_Metrics_df.loc[var, 'SCORR'] = spatial_corrs
        S_Metrics_df.loc[var, 'SRMSE'] = spatial_rmses
        S_Metrics_df.loc[var, 'SBIAS'] = spatial_bias
    S_Metrics_df.to_excel(f"LSMvar_mean_Spatial_metrics_{EXP}.xlsx")
    # 输出结果
    print("Spatial Metrics DataFrame:")
    